# Projet Métabolisme - Partie 2
## Analyse des flux métaboliques sur un modèle genome-scale avec MetExplore

### 1. Dans cobrapy, Créer le modèle à partir du réseau SBML téléchargé précédemment

In [31]:
import cobra
model = cobra.io.read_sbml_model("ralsto_metexplore3.xml")

Adding exchange reaction EX_hdcea_b with default bounds for boundary metabolite: hdcea_b.
Adding exchange reaction EX_RSc1944_b with default bounds for boundary metabolite: RSc1944_b.
Adding exchange reaction EX_3oochslac_b with default bounds for boundary metabolite: 3oochslac_b.
Adding exchange reaction EX_glu_D_b with default bounds for boundary metabolite: glu_D_b.
Adding exchange reaction EX_RipS4_b with default bounds for boundary metabolite: RipS4_b.
Adding exchange reaction EX_fecrm_b with default bounds for boundary metabolite: fecrm_b.
Adding exchange reaction EX_ni2_b with default bounds for boundary metabolite: ni2_b.
Adding exchange reaction EX_RipK_b with default bounds for boundary metabolite: RipK_b.
Adding exchange reaction EX_berb_b with default bounds for boundary metabolite: berb_b.
Adding exchange reaction EX_tartr_L_b with default bounds for boundary metabolite: tartr_L_b.
Adding exchange reaction EX_octa_b with default bounds for boundary metabolite: octa_b.
Addi

In [32]:
model

Name,_bc9d1403_3bf5_4994_8369_f3c28ce8995a
Memory address,115079010
Number of metabolites,2574
Number of reactions,3097
Number of genes,2219
Number of groups,253
Objective expression,0
Compartments,"boundary, extracellular, cytoplasm, periplasm"


### 2- Combien de réactions, métabolites, gènes contient le réseau ?

In [33]:
print("Le nombre de réactions: ", len(model.reactions))
print("Le nombre de métabolites: ", len(model.metabolites))
print("Le nombre de gènes: ", len(model.genes))

Le nombre de réactions:  3097
Le nombre de métabolites:  2574
Le nombre de gènes:  2219


Le réseau SBML de Ralstonia solanacearum contient 3097 réactions, 2574 métabolites et 2219 gènes 

### 3. Indiquer la réaction BIOMASS comme fonction objectif

In [34]:
# Récupération de la réaction de biomasse par son identifiant
biomass_rxn = model.reactions.get_by_id("BIOMASS")
biomass_rxn

Reaction identifier,BIOMASS
Name,Biomass_formation
Memory address,0x1172cc920
Stoichiometry,0.000223 10fthf_c + 2.6e-05 2fe2s_c + 0.00026 4fe4s_c + 0.01068 Oantigen_core_kdo_lipidA_RS_e + 0.0094 acglucan_p + 0.000223 adocbl_c + 0.6411 alatrna_c + 0.000223 amet_c + 0.24141 argtrna_c +... 0.000223 10_Formyltetrahydrofolate + 2.6e-05 2Fe_2S__iron_sulfur_cluster + 0.00026 4Fe_4S__iron_sulfur_cluster + 0.01068 Oantigen_core_kdo_lipidA_Ralstonia_solanacearum + 0.0094...
GPR,e0002
Lower bound,0.0
Upper bound,99999.0


In [35]:
# Définition de la réaction BIOMASS comme fonction objective
model.objective = biomass_rxn
model.objective

In [36]:
#vérification
from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(model)

{<Reaction BIOMASS at 0x1172cc920>: 1.0}

In [37]:
model

Name,_bc9d1403_3bf5_4994_8369_f3c28ce8995a
Memory address,115079010
Number of metabolites,2574
Number of reactions,3097
Number of genes,2219
Number of groups,253
Objective expression,1.0*BIOMASS - 1.0*BIOMASS_reverse_69053
Compartments,"boundary, extracellular, cytoplasm, periplasm"


La réaction BIOMASS a bien été définie comme fonction objectif du modèle.
Elle est associée à un coefficient de 1.0, ce qui indique que le solveur cherche à maximiser la production de biomasse.

### 4. Mettre à 0 la “lower bound” de toutes réactions d’échange du modèle.

In [38]:
for rxn in model.exchanges:
    rxn.lower_bound = 0.0

In [39]:
# Vérification: Affichage de la 3ème réaction d'échange du modèle 
model.exchanges[2]

Reaction identifier,EX_3oochslac_b
Name,EX_3oochslac_b
Memory address,0x1156a2b10
Stoichiometry,3oochslac_b --> N__3_Oxooctanoyl_homoserine_lactone -->
GPR,
Lower bound,0.0
Upper bound,1000.0


### 5. Fixer les “lower bound” et “upper bound” des réactions d’échange faisant intervenir les métabolites ci-dessous respectivement à -1000 et à 1000

In [40]:
# Liste des métabolites pour lesquels l'échange est autorisé
metabolites = [
    "h2o_b", "h_b", "k_b", "pi_b", "na1_b", "nh4_b", "so4_b", "mg2_b",
    "cl_b", "fe2_b", "fe3_b", "cobalt2_b", "mn2_b", "mobd_b", "o2_b", "co2_b"
]

# Modification des bornes “lower bound” et “upper bound” 
for met in metabolites:
    rxn_id = "EX_" + met      
    rxn = model.reactions.get_by_id(rxn_id)
    rxn.lower_bound = -1000.0
    rxn.upper_bound = 1000.0

# Vérification 
for met in metabolites:
    rxn_id = "EX_" + met
    rxn = model.reactions.get_by_id(rxn_id)
    print(rxn.id, " lb =", rxn.lower_bound, " ub =", rxn.upper_bound)


EX_h2o_b  lb = -1000.0  ub = 1000.0
EX_h_b  lb = -1000.0  ub = 1000.0
EX_k_b  lb = -1000.0  ub = 1000.0
EX_pi_b  lb = -1000.0  ub = 1000.0
EX_na1_b  lb = -1000.0  ub = 1000.0
EX_nh4_b  lb = -1000.0  ub = 1000.0
EX_so4_b  lb = -1000.0  ub = 1000.0
EX_mg2_b  lb = -1000.0  ub = 1000.0
EX_cl_b  lb = -1000.0  ub = 1000.0
EX_fe2_b  lb = -1000.0  ub = 1000.0
EX_fe3_b  lb = -1000.0  ub = 1000.0
EX_cobalt2_b  lb = -1000.0  ub = 1000.0
EX_mn2_b  lb = -1000.0  ub = 1000.0
EX_mobd_b  lb = -1000.0  ub = 1000.0
EX_o2_b  lb = -1000.0  ub = 1000.0
EX_co2_b  lb = -1000.0  ub = 1000.0


### 6. Pourquoi met-on le flux lower bound à 0 pour toutes les réactions d’échange sauf pour les réactions faisant intervenir certains métabolites ?

On met le lower bound à 0 pour toutes les réactions d’échange afin de bloquer l’import de tous les métabolites et de partir d’un milieu sans apport externe. On réactive ensuite l’import uniquement pour les métabolites que l’on souhaite autoriser dans le milieu étudié. Cette démarche permet de définir précisément les conditions environnementales du modèle et de contrôler les métabolites que le modèle peut importer, afin d’analyser les flux métaboliques dans le milieu étudié.

### 7. Fixer les flux des réactions suivantes aux valeurs suivantes : NGAME : 8.39 (réaction correspondant à un flux de consommation d’ATP correspondant à une maintenance de la cellule) FEOXpp : 0 (cette réaction quand elle est active peut amener à produire de l’ATP “gratuitement”)

In [41]:
# Réaction de maintenance de la cellule
ngame_rxn = model.reactions.get_by_id("NGAME")

# On fixe le flux à 8.39 en mettant LB = UB = 8.39
ngame_rxn.lower_bound = 8.39
ngame_rxn.upper_bound = 8.39

print(ngame_rxn.id, " lb =", ngame_rxn.lower_bound, " ub =", ngame_rxn.upper_bound)


NGAME  lb = 8.39  ub = 8.39


In [42]:
# Réaction qui pourrait produire de l'ATP "gratuitement"
feox_rxn = model.reactions.get_by_id("FEOXpp")

# On impose un flux nul : LB = 0, UB = 0
feox_rxn.lower_bound = 0.0
feox_rxn.upper_bound = 0.0

print(feox_rxn.id, " lb =", feox_rxn.lower_bound, " ub =", feox_rxn.upper_bound)


FEOXpp  lb = 0.0  ub = 0.0


### 8. Calculer le taux de croissance dans ces conditions.

In [43]:
model.optimize()

/Users/hawabalde/.cobraenv/lib/python3.12/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


<Solution infeasible at 0x114e40680>

In [44]:
solution = model.optimize()
# Taux de croissance
growth_rate = solution.objective_value
growth_rate

0.0

Le taux de croissance dans ces conditions est égale à 0.

#### Quels sont les métabolites importés qui permettent ce taux de croissance optimal ? Quels sont les métabolites excrétés ?

Dans ces conditions, le modèle renvoie un statut infeasible et la valeur de la fonction objectif BIOMASS, qui correspond au taux de croissance, est égale à 0. Cela signifie qu’aucune croissance n’est possible avec les contraintes imposées. Par conséquent, aucun métabolite n’est importé ni excrété, car aucun flux d’échange ne permet au modèle d’assurer la production de biomasse.

### 9. Fixer le flux lower bound et upper des réactions de transports des métabolites ci-dessous, aux valeurs données ci-dessous 

In [45]:
# Flux des réactions de transport des métabolites
transport_fluxes = {
    "glu_L_b": -8.25,
    "3ohpame_b": 1.5e-4,
    "EPS_b": 0.0062,
    "ptrc_b": 0.28,
    "Tek_28kd_b": 2.7e-4,
    "etle_b": 0.0129,
}

# Application des valeurs aux réactions de transport correspondantes
for met, value in transport_fluxes.items():
    rxn_id = "EX_" + met
    rxn = model.reactions.get_by_id(rxn_id)
    rxn.lower_bound = value
    rxn.upper_bound = value
    print(rxn.id, " lb =", rxn.lower_bound, " ub =", rxn.upper_bound)


EX_glu_L_b  lb = -8.25  ub = -8.25
EX_3ohpame_b  lb = 0.00015  ub = 0.00015
EX_EPS_b  lb = 0.0062  ub = 0.0062
EX_ptrc_b  lb = 0.28  ub = 0.28
EX_Tek_28kd_b  lb = 0.00027  ub = 0.00027
EX_etle_b  lb = 0.0129  ub = 0.0129


### Quel est le taux de croissance dans ces conditions ? Que pouvez-vous en dire ?

In [46]:
solution = model.optimize()
growth_rate = solution.objective_value
growth_rate


0.5119279935093645

In [47]:
model.summary()


Metabolite,Reaction,Flux,C-Number,C-Flux
cl_b,EX_cl_b,0.002665,0,0.00%
cobalt2_b,EX_cobalt2_b,0.000127,0,0.00%
fe2_b,EX_fe2_b,0.004225,0,0.00%
glu_L_b,EX_glu_L_b,8.25,0,0.00%
h_b,EX_h_b,14.44,0,0.00%
k_b,EX_k_b,0.09992,0,0.00%
mg2_b,EX_mg2_b,0.004441,0,0.00%
mn2_b,EX_mn2_b,0.0003537,0,0.00%
mobd_b,EX_mobd_b,6.604E-05,0,0.00%
o2_b,EX_o2_b,13.4,0,0.00%


Dans ces conditions, le modèle prédit un taux de croissance de 0,512 h⁻¹.

Contrairement au cas précédent, où le modèle était infaisable et la croissance nulle, l’imposition des flux de transport mesurés expérimentalement (Peyraud et al., 2016) permet ici d’obtenir une solution faisable et une croissance positive. Cela montre que les apports et sécrétions imposés fournissent au réseau métabolique les ressources nécessaires à la production de biomasse.

Le résumé des flux met en évidence l’import de métabolites essentiels tels que le glucose, l’oxygène, le phosphate et certains ions, ainsi que l’excrétion de composés métaboliques comme le CO₂, l’eau, l’ammonium ou les EPS, ce qui est cohérent avec une cellule en croissance active.

Ces résultats confirment que les conditions expérimentales définies permettent au modèle d’assurer une croissance.

### 10- Expérimentalement, Ralstonia solanacearum pousse à 0.28h-1 .Calculer la valeur du flux de la réaction NGAME pour atteindre ce taux de croissance en FBA. Quelle est cettevaleur ? Trouver la méthode la plus parcimonieuse pour trouver cette valeur et décrire la.

In [48]:
with model as m:  # copie m du modèle
    # On récupère BIOMASS et NGAME dans la copie m
    biomass_rxn_m = m.reactions.get_by_id("BIOMASS")
    ngame_rxn_m = m.reactions.get_by_id("NGAME")

    # 1) On libère NGAME
    ngame_rxn_m.lower_bound = 0.0
    ngame_rxn_m.upper_bound = 1000.0

    # 2) On fixe la croissance à la valeur expérimentale 0.28 h-1
    biomass_rxn_m.lower_bound = 0.28
    biomass_rxn_m.upper_bound = 0.28

    # 3) On definit NGAME comme fonction objective et on minimise son flux
    m.objective = ngame_rxn_m
    m.objective_direction = "min"

    sol = m.optimize()
    print("Flux NGAME :", sol.objective_value)


Flux NGAME : 0.0


Pour retrouver expérimentalement une croissance de 0,28 h⁻¹, cette valeur a été imposée comme contrainte sur la réaction BIOMASS, puis le flux de NGAME a été minimisé. 
C’est le principe d’une approche parcimonieuse : parmi toutes les solutions possibles qui atteignent la croissance voulue, on cherche celle qui utilise le moins d’ATP de maintenance.

Avec cette méthode, la valeur trouvée pour NGAME est : 0.0
Cela signifie que, dans les conditions définies dans le modèle, une croissance de 0,28 h⁻¹ peut être atteinte sans nécessiter de flux supplémentaire de maintenance. Le modèle parvient donc à satisfaire les besoins énergétiques liés à cette croissance grâce aux autres réactions métaboliques actives, sans activer la réaction NGAME.

### 11-  Sauver les valeurs de flux dans un fichier tabulé (attention, pensez à ajouter les suffixesR_! ) et les mapper sur les arêtes du sous-réseau métabolique du dessin de la glycolyse de Ralstonia dans metexplore

In [49]:
solution = model.optimize()
print("Growth rate =", solution.objective_value)


Growth rate = 0.5119279935093696


In [50]:
import pandas as pd

fluxes = pd.DataFrame({
    # R_ au DÉBUT des réactions
    "reaction": ["R_" + rxn.id for rxn in model.reactions],
    "flux": [solution.fluxes[rxn.id] for rxn in model.reactions],
})

fluxes.to_csv(
    "reaction_fluxes.tab",
    sep="\t",
    index=False
)

fluxes.head()


,reaction,flux
0,R_EX_hdcea_b,0.0
1,R_EX_RSc1944_b,0.0
2,R_EX_3oochslac_b,0.0
3,R_EX_glu_D_b,0.0
4,R_EX_RipS4_b,0.0


### Mapping des flux métaboliques sur la glycolyse de Ralstonia solanacearum

![glycolysis flux mapping](glycolysis_flux_mapping.png)

Les flux calculés pour obtenir la croissance optimale de Ralstonia solanacearum ont été mappés sur le sous-réseau de la glycolyse.
L’épaisseur des arêtes reflète l’intensité des flux, mettant en évidence les réactions les plus actives dans ces conditions de croissance.

### 12-A) Déterminer avec cobrapy les gènes essentiels qui permettent cette croissance optimale. Les mapper sur MetExplore 3

In [51]:
# Optimisation pour récupérer la croissance optimale (BIOMASS)
solution = model.optimize()
opt_growth = solution.objective_value
print("Croissance optimale (BIOMASS) =", opt_growth)

Croissance optimale (BIOMASS) = 0.5119279935093664


In [52]:
# Single gene deletion
from cobra.flux_analysis import single_gene_deletion
gene_del = single_gene_deletion(model)
gene_del.head()

,ids,growth,status
0,{RSp0283},0.511928,optimal
1,{e0069},0.511928,optimal
2,{d0188},0.511928,optimal
3,{RSc0920},0.511928,optimal
4,{RSc1601},0.503506,optimal


In [54]:
# Définir un seuil : gène essentiel si la croissance devient ~0
# (ici: <= 1e-9 * croissance optimale)
threshold = 1e-9 * opt_growth
essential_genes = gene_del[gene_del["growth"] <= threshold].copy()

print("Nombre de gènes essentiels :", essential_genes.shape[0])
essential_genes.head(10)


Nombre de gènes essentiels : 197


,ids,growth,status
18,{e0193},-3.090061e-17,optimal
39,{e0199},2.893997e-17,optimal
55,{RSc2848},1.963648e-17,optimal
76,{RSc2204},5.031505e-16,optimal
90,{RSc1163},-1.039729e-15,optimal
91,{RSc1981},-3.397431e-18,optimal
105,{d0055},6.180122e-17,optimal
115,{RSc1021},-3.340248e-16,optimal
125,{RSc0390},7.515952e-14,optimal
143,{RSc2532},1.376942e-16,optimal


In [55]:
# Extraire les IDs de gènes
essential_gene_ids = []
for gset in essential_genes["ids"]:
    for gid in gset:
        essential_gene_ids.append(gid)

# enlever doublons + trier pour que ce soit propre
essential_gene_ids = sorted(set(essential_gene_ids))
# Affichage
essential_gene_ids[:10], len(essential_gene_ids)

(['RS04716',
  'RSc0100',
  'RSc0109',
  'RSc0111',
  'RSc0139',
  'RSc0157',
  'RSc0266',
  'RSc0311',
  'RSc0322',
  'RSc0323'],
 197)

In [56]:
# Export fichier Pour les mapper sur MetExplore 3
import pandas as pd

essential_genes_csv = pd.DataFrame({
    "gene": essential_gene_ids
})

essential_genes_csv.to_csv(
    "essential_genes.csv",
    index=False
)


La fonction single_gene_deletion consiste à supprimer chaque gène du modèle, un par un, puis à recalculer le taux de croissance.
Si la suppression d’un gène entraîne une croissance nulle ou quasi nulle, cela signifie que ce gène est indispensable au fonctionnement du réseau métabolique dans les conditions étudiées : il est alors considéré comme essentiel.

Dans notre cas, on compare le taux de croissance après suppression de chaque gène au taux de croissance optimal. Les gènes pour lesquels la croissance devient inférieure à un seuil très faible sont identifiés comme gènes essentiels pour cette croissance optimale.

### 12-B) Quelles sont les voies métaboliques contenant ces gènes métaboliques ? Les lister dans le notebook et y copier l’image du résultat du mapping montrant les voies métaboliques contenant les gènes.

#### Voies métaboliques contenant les gènes essentiels

Le mapping des gènes essentiels réalisé sur MetExplore montre que ces gènes sont impliqués dans **46 voies métaboliques** différentes qui sont:

-Glycine_and_serine_degradation  
-Putrescine_degradation  
-Pyridoxal_5__phosphate_biosynthesis  
-Purine_biosynthesis  
-Fatty_acid_biosynthesis  
-Thiamin_biosynthesis  
-Flavin_biosynthesis  
-Transport_ion_substrate  
-IAA_biosynthesis  
-Coenzyme_A_biosynthesis  
-Folate_biosynthesis  
-Tryptophan_biosynthesis  
-Valine_degradation  
-Heme_biosynthesis  
-Histamine_degradation  
-Alternate_carbon_metabolism  
-Sink  
-Propanoate_metabolism  
-Ubiquinone_biosynthesis  
-Transport_carbon_substrate  
-Oxidative_phosphorylation  
-Exchange_ion_substrate  
-Glycerophospholipid_metabolism  
-Ethanol_degradation  
-Molybdenum_cofactor_biosynthesis  
-Polyamine_biosynthesis  
-Glutathione_biosynthesis  
-Glycine_and_serine_biosynthesis  
-Tyrosine__tryptophan__and_phenylalanine_metabolism  
-Isoprenoid_biosynthesis  
-Biotin_biosynthesis  
-NAD_biosynthesis  
-Pyrimidine_biosynthesis  
-Glycerol_metabolism  
-B_alanine_biosynthesis  
-Glycogen_biosynthesis  
-Adenosylcobalamin_biosynthesis  
-Exchange_carbon_substrate  
-Lipoate_biosynthesis  
-Assembly_and_repair_of_iron_sulfur_proteins  
-Butanoate_metabolism  
-S_adenosyl_L_methionine_cycle  
-Biomass_equation  
-Lipopolysaccharide_biosynthesis  
-Peptidoglycan_biosynthesis  
-UDP_D_Xylose_and_UDP_D_glucuronate_biosynthesis  

#### Représentation graphique des voies métaboliques
![Mapping des gènes essentiels de Ralstonia solanacearum](mapping_genes_essentiels.jpg)

Le graphique présente les voies métaboliques contenant les gènes essentiels mappés, en comparant pour chaque voie le nombre total de gènes associés et le nombre de gènes essentiels mappés. Les voies sont ordonnées par significativité décroissante selon la p-value ajustée (Benjamini–Hochberg).

Parmi les voies les plus significatives figurent notamment celles impliquées dans la biosynthèse des lipopolysaccharides, de l’adénosylcobalamine, de l’ubiquinone, des folates, du peptidoglycane, des isoprénoïdes ainsi que du cofacteur au molybdène. Ces voies correspondent principalement à des processus essentiels à la croissance et à la survie de la bactérie.
